In [4]:
import pandas as pd
from citipy import citipy
import requests
from config import weather_api_key, g_key
from datetime import datetime
import numpy as np

In [5]:
lat=np.random.uniform(-90,90,1500)
lng=np.random.uniform(-90,90,1500)
lat_lng=zip(lat,lng)
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lng)

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

606

In [9]:
city_data = []


# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")



# Loop through all the cities in the list.
for i, city in enumerate(cities):

    if i%100==0:
        print(f"-----{i} completed.------")
    city_url = "http://api.openweathermap.org/data/2.5/weather?units=imperial&appid=" + weather_api_key + "&q=" + city

    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_conditions = city_weather["weather"][0]["description"]
        try:
            city_rain = city_weather["rain"]["3h"]
        except KeyError:
            city_rain = 0
        try:
            city_snow = city_weather["snow"]["3h"]
        except KeyError:
            city_snow = 0
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Current Conditions":city_conditions,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Rainfall": city_rain,
                          "Snowfall": city_snow,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except KeyError:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
-----0 completed.------
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
-----100 completed.------
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
-----200 completed.------
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
-----300 completed.------
City not found. Skipping..

In [10]:
city_df = pd.DataFrame(city_data)
city_df.to_csv("Data/weatherpy_challenge.csv", index_label="City_ID")

In [11]:
#Count cities that had some type of precipitation
precip= ((city_df['Rainfall']!=0) | (city_df['Snowfall']!=0)).sum()
precip

68